<a href="https://colab.research.google.com/github/mshojaei77/Awesome-Fine-tuning/blob/main/combine_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyarrow==11.0.0
!pip install datasets==2.14.5

In [13]:

from huggingface_hub import notebook_login

notebook_login()


In [6]:
from datasets import load_dataset

dataset1 = load_dataset("databricks/databricks-dolly-15k", split="train")
print(dataset1)

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})


In [7]:
dataset2 = load_dataset("yahma/alpaca-cleaned", split="train")
print(dataset2)

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 51760
})


In [15]:
dataset3 = load_dataset("lmsys/lmsys-chat-1m", split="train")
print(dataset3)

Dataset({
    features: ['conversation_id', 'model', 'conversation', 'turn', 'language', 'openai_moderation', 'redacted'],
    num_rows: 1000000
})


In [16]:
from datasets import DatasetDict, concatenate_datasets

# Transform dataset1
dataset1_transformed = dataset1.map(lambda x: {"instruction": x["instruction"], "context": x["context"], "response": x["response"]})
dataset1_transformed = dataset1_transformed.remove_columns([col for col in dataset1_transformed.column_names if col not in ["instruction", "context", "response"]])

# Transform dataset2
dataset2_transformed = dataset2.map(lambda x: {"instruction": x["instruction"], "context": x["input"], "response": x["output"]})
dataset2_transformed = dataset2_transformed.remove_columns([col for col in dataset2_transformed.column_names if col not in ["instruction", "context", "response"]])

# Transform dataset3
def transform_dataset3(example):
    # Extract instruction and response from the conversation
    conversation = example['conversation']
    instruction = conversation[0]['content'] if len(conversation) > 0 else ""
    response = conversation[1]['content'] if len(conversation) > 1 else ""
    context = ""
    return {"instruction": instruction, "context": context, "response": response}

dataset3_transformed = dataset3.map(transform_dataset3)
dataset3_transformed = dataset3_transformed.remove_columns([col for col in dataset3_transformed.column_names if col not in ["instruction", "context", "response"]])

# Combine the datasets
combined_dataset = concatenate_datasets([dataset1_transformed, dataset2_transformed, dataset3_transformed])

# Print the combined dataset to verify
print(combined_dataset)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'context', 'response'],
    num_rows: 1066771
})


In [17]:
# Push the combined dataset to Hugging Face
combined_dataset.push_to_hub("mshojaei77/Dolly_Alpaca_Lmsys")

Pushing dataset shards to the dataset hub:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/356 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/356 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/356 [00:00<?, ?ba/s]